In [1]:
!pip install googletrans==3.1.0a0

In [2]:
!pip install googletrans==3.1.0a0 Pillow pyperclip gTTS SpeechRecognition

In [5]:
# 1. Install necessary libraries
!pip install gradio googletrans==3.1.0a0 gTTS SpeechRecognition pydub

import gradio as gr
from googletrans import Translator, LANGUAGES
from gtts import gTTS
import speech_recognition as sr
import os

# 2. Define the Logic
def translator_logic(text_input, audio_input, target_language):
    translator = Translator()
    detected_text = ""

    # --- STEP 1: INPUT HANDLING (Text vs Voice) ---
    if audio_input is not None:
        # User used the microphone
        r = sr.Recognizer()
        with sr.AudioFile(audio_input) as source:
            audio_data = r.record(source)
            try:
                # Convert speech to text (English default for input as per your original code)
                text_input = r.recognize_google(audio_data)
                detected_text = f"🎤 Voice Detected: {text_input}"
            except sr.UnknownValueError:
                return "Could not understand audio", "Error", None
            except sr.RequestError:
                return "Speech service unavailable", "Error", None
    elif text_input:
        detected_text = f"📝 Text Input: {text_input}"
    else:
        return "Please enter text or record audio.", "", None

    # --- STEP 2: TRANSLATION ---
    try:
        # Translate the text
        trans_obj = translator.translate(text_input, dest=target_language)
        translated_text = trans_obj.text
        dest_code = trans_obj.dest # Get the language code (e.g., 'es' for Spanish)
    except Exception as e:
        return f"Translation Error: {str(e)}", "Error", None

    # --- STEP 3: TEXT TO SPEECH ---
    output_audio_path = "output_audio.mp3"
    try:
        # Generate audio from translated text
        tts = gTTS(translated_text, lang=dest_code)
        tts.save(output_audio_path)
    except ValueError:
        # Fallback if language not supported by gTTS
        output_audio_path = None
    except Exception as e:
        print(f"TTS Error: {e}")
        output_audio_path = None

    return detected_text, translated_text, output_audio_path

# 3. Create the Web Interface
# Get list of languages from googletrans
lang_options = list(LANGUAGES.values())

# Build the App
app = gr.Interface(
    fn=translator_logic,
    inputs=[
        gr.Textbox(label="Enter Text Here", placeholder="Type something..."),
        gr.Audio(sources=["microphone"], type="filepath", label="Or Record Voice (English)"),
        gr.Dropdown(choices=lang_options, value='spanish', label="Select Target Language")
    ],
    outputs=[
        gr.Textbox(label="Input Status"),
        gr.Textbox(label="Translated Text", show_copy_button=True), # Includes Copy button
        gr.Audio(label="Listen to Translation")
    ],
    title="Simple AI Language Translator",
    description="Type text OR use your microphone. Select a language to translate and listen to the result.",
    theme="default"
)

# 4. Launch
app.launch()

IMPORTANT: You are using gradio version 4.19.1, however version 4.44.1 is available, please upgrade.
--------
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f84b057af0e542d6d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
